在inference的过程中，testing数据集中存在一个sid包含多个view的情况，即不同的view应该对应相同的结果。    
在训练阶段，同一个sid的不同views对应的label都应该是训练的samples--因为希望模型从不同的view中都能学到该疾病的表示。  
但是在inference阶段，一个sid对应的多张image应该被综合考虑。一个sid应该是一个sample。当不同view的prediction结果相反时，取预测结果概率较高的。


In [1]:
import pandas as pd

## inference sample

##### load data

In [51]:
# loading current training 
current_testing = pd.read_csv("../data/mimic-cxr-train/P10_12_test_2_29_labels14_binary.csv")
whole_dataset = pd.read_csv("/home_data/home/v-liudsh/coding/constrastive_P/diagnosisP/exchange/Fine-Grained_Features_Alignment_via_Constrastive_Learning_old/data/mimic-cxr-2.0.0-chexpert.csv", index_col=0)
split_dataset = pd.read_csv("/home_data/home/v-liudsh/coding/constrastive_P/diagnosisP/exchange/Fine-Grained_Features_Alignment_via_Constrastive_Learning_old/data/mimic-cxr-2.0.0-split.csv", index_col = 0)
testing  = split_dataset[split_dataset["split"] == "test"]
# get sid for repeated sample
def custom_func(x):
  temp = x.split("/")
  sid = temp[-2]
  assert "s" in sid
  return sid
  
# current_testing["sid"] = current_testing["BiomedClip_img_tensor_path"].apply(custom_func)

In [52]:
current_testing.head()
sid = current_testing['sid']
the_number_of_all = len(sid)
print(f"the total number of testing dataset: {the_number_of_all}")
print(f"the number of unique sid in current testing dataset: {len(sid.unique())}")

the total number of testing dataset: 1175
the number of unique sid in current testing dataset: 783


currently, only 783 samples are been used for evaluation.  
the testing dataset need to be expanded.   
using all of testing sample indicated in `split.csv`

In [53]:
expand_testing = testing[(testing['subject_id'] > 13000000) & (testing['subject_id'] < 20000000)]
expand_study_id = expand_testing["study_id"]


In [54]:
def justify_sameSid_same_label():
  unique_sid =  current_testing.sid.unique()
  for sid in unique_sid:
    temp = current_testing[current_testing['sid'] == sid]
    temp = set(temp['binary_label'])
    assert len(temp) == 1
  print("\033[32mjustify pass:\033[0m in the dataset, all of sample with the same id using the same labels.")
    
justify_sameSid_same_label()
    

justify pass: in the dataset, all of sample with the same id using the same labels.


In [55]:
whole_dataset

,study_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
subject_id,,,,,,,,,,,,,,,
10000032,50414267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
10000032,53189527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
10000032,53911762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
10000032,56699142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
10000764,57375967,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19999442,58708861,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
19999733,57132437,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
19999987,55368167,1.0,-1.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN


##### expand current testing dataset

In [56]:
# extract row data which will be used to expand testing dataset
expand_in_whole_dataset = whole_dataset[whole_dataset['study_id'].isin(expand_study_id)]
a = set(current_testing.sid)
b = set(expand_in_whole_dataset.study_id)
print("the intersection between current testing dataset and expanding dataset:", a.intersection(b))

the intersection between current testing dataset and expanding dataset: set()


<font color=red> ToDo</font>  
<li>
<input type=checkbox> 扩充测试数据集  
</li>
<li>
<input type=checkbox> 调整测试代码 -- 多view 同sample的case 
</li>
<li>
<input type=checkbox> 在当前的测试数据中有多少同sample但是结果不一样的情况（百分比）
</li>

### model inference

##### 当前testing数据集中，同sample存在预测结果矛盾的情况

In [106]:
res = torch.load("/home_data/home/v-liudsh/coding/constrastive_P/diagnosisP/exchange/Fine-Grained_Features_Alignment_via_Constrastive_Learning/src/output/dump_prediction_output/biomedclip_None_False_False_NA_False_False_task_balance/tensor.pth")
res_1 = res["predictions"]
res_1 = res_1.view(1175, 14 , 3)
softmax_output = torch.softmax(res_1, dim=-1)
max_indices = torch.argmax(softmax_output, dim=-1)
res_2 = res["labels"]
 
print(max_indices.shape)  # 输出最大值索引的形状
print(res["labels"])

torch.Size([1175, 14])
tensor([[2., 2., 2.,  ..., 2., 2., 0.],
        [2., 1., 2.,  ..., 2., 2., 0.],
        [2., 2., 2.,  ..., 2., 2., 2.],
        ...,
        [2., 2., 2.,  ..., 0., 2., 2.],
        [2., 2., 2.,  ..., 2., 1., 0.],
        [2., 2., 2.,  ..., 2., 2., 0.]])


In [111]:
current_unique_id = current_testing.sid.unique()

In [132]:
pre_one = -1
dic = {}
for index, row in current_testing.iterrows():
  sid = row.sid
  if sid != pre_one:
    pre_one = sid
    assert sid not in dic
    dic[sid] = 1
  else:
      dic[sid] += 1
  


In [145]:
dic_pred_res = {}
index = 0
for i, j in dic.items(): 
  dic_pred_res[i] = max_indices [index : index + j, :]
  index = index + j
  

In [159]:
dul_list = []
dul_list_1 = []
max_dul = -1
for i, j in dic_pred_res.items():
  if j.shape[0] > 1:
    # print(i, j )
    dul_list.append(i)
    max_dul = j.shape[0] if j.shape[0] > max_dul else max_dul
    
    if (len(set(j)) > 1):
      dul_list_1.append(i)
  
print(len(dul_list))
print(max_dul)
print(len(dul_list_1))

319
4
319


在当前的数据集中，有319个sample存在多个view，一个sample最多有四个view，在这319个sample中每一个prediction的结果都不一样   
从当前的数据看，predication产生矛盾的sample比例是  <b><font color=red>319/783 = 40.74%</font></b>

##### 扩充当前testing数据集 （training的数据也需要相应的调整--label）
在当前的testing数据集中，存在数据分配不均的情况，导致testing中的数据分布不具有代表性

In [165]:
current_testing.head(1).label.values

array(["{'Atelectasis': 1, 'Cardiomegaly': 1, 'Consolidation': 1, 'Edema': 1, 'Enlarged Cardiomediastinum': 1, 'Fracture': 1, 'Lung Lesion': 1, 'Lung Opacity': 1, 'No Finding': 0, 'Pleural Effusion': 1, 'Pleural Other': 1, 'Pneumonia': 1, 'Pneumothorax': 1}"],
      dtype=object)

In [166]:
current_testing

,label,img_path0,train_label,file_path,split,BiomedClip_img_tensor_path,Clip_img_tensor_path,sid,test_14_labels,Biomed_img_tensor_path,Biovil_img_tensor_path,binary_label
0,"{'Atelectasis': 1, 'Cardiomegaly': 1, 'Consoli...",D:/project_x_ray_CLIP/data/physionet.org/files...,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]",D:/project_x_ray_CLIP/data/physionet.org/files...,test,/public_bme/data/lds/p10/p10032725/s50331901/6...,/public_bme/data/lds/p10/p10032725/s50331901/6...,50331901,"[2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 0]",/public_bme/data/lds/p10/p10032725/s50331901/6...,/public_bme/data/lds/p10/p10032725/s50331901/6...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]"
1,"{'Atelectasis': 1, 'Cardiomegaly': 1, 'Consoli...",D:/project_x_ray_CLIP/data/physionet.org/files...,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]",D:/project_x_ray_CLIP/data/physionet.org/files...,test,/public_bme/data/lds/p10/p10032725/s55504914/f...,/public_bme/data/lds/p10/p10032725/s55504914/f...,55504914,"[2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 0]",/public_bme/data/lds/p10/p10032725/s55504914/f...,/public_bme/data/lds/p10/p10032725/s55504914/f...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]"
2,"{'Atelectasis': 1, 'Cardiomegaly': 1, 'Consoli...",D:/project_x_ray_CLIP/data/physionet.org/files...,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]",D:/project_x_ray_CLIP/data/physionet.org/files...,test,/public_bme/data/lds/p10/p10046166/s50051329/4...,/public_bme/data/lds/p10/p10046166/s50051329/4...,50051329,"[2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2]",/public_bme/data/lds/p10/p10046166/s50051329/4...,/public_bme/data/lds/p10/p10046166/s50051329/4...,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1]"
3,"{'Atelectasis': 1, 'Cardiomegaly': 1, 'Consoli...",D:/project_x_ray_CLIP/data/physionet.org/files...,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]",D:/project_x_ray_CLIP/data/physionet.org/files...,test,/public_bme/data/lds/p10/p10046166/s50051329/a...,/public_bme/data/lds/p10/p10046166/s50051329/a...,50051329,"[2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2]",/public_bme/data/lds/p10/p10046166/s50051329/a...,/public_bme/data/lds/p10/p10046166/s50051329/a...,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1]"
4,"{'Atelectasis': 1, 'Cardiomegaly': 1, 'Consoli...",D:/project_x_ray_CLIP/data/physionet.org/files...,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]",D:/project_x_ray_CLIP/data/physionet.org/files...,test,/public_bme/data/lds/p10/p10046166/s51738740/3...,/public_bme/data/lds/p10/p10046166/s51738740/3...,51738740,"[2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2]",/public_bme/data/lds/p10/p10046166/s51738740/3...,/public_bme/data/lds/p10/p10046166/s51738740/3...,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1]"
...,...,...,...,...,...,...,...,...,...,...,...,...
1170,"{'Atelectasis': 2, 'Cardiomegaly': 0, 'Consoli...",D:/project_x_ray_CLIP/data/physionet.org/files...,"[2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2]",D:/project_x_ray_CLIP/data/physionet.org/files...,test,/public_bme/data/lds/p12/p12963531/s59505688/4...,/public_bme/data/lds/p12/p12963531/s59505688/4...,59505688,"[2, 0, 2, 0, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2]",/public_bme/data/lds/p12/p12963531/s59505688/4...,/public_bme/data/lds/p12/p12963531/s59505688/4...,"[1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]"
1171,"{'Atelectasis': 2, 'Cardiomegaly': 0, 'Consoli...",D:/project_x_ray_CLIP/data/physionet.org/files...,"[2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2]",D:/project_x_ray_CLIP/data/physionet.org/files...,test,/public_bme/data/lds/p12/p12963531/s59505688/5...,/public_bme/data/lds/p12/p12963531/s59505688/5...,59505688,"[2, 0, 2, 0, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2]",/public_bme/data/lds/p12/p12963531/s59505688/5...,/public_bme/data/lds/p12/p12963531/s59505688/5...,"[1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]"
1172,"{'Atelectasis': 2, 'Cardiomegaly': 2, 'Consoli...",D:/project_x_ray_CLIP/data/physionet.org/files...,"[2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2]",D:/project_x_ray_CLIP/data/physionet.org/files...,test,/public_bme/data/lds/p12/p12966004/s55553875/d...,/public_bme/data/

In [168]:
train = pd.read_csv("/home_data/home/v-liudsh/coding/constrastive_P/diagnosisP/exchange/Fine-Grained_Features_Alignment_via_Constrastive_Learning/data/mimic-cxr-train/P10_12_train_2_29_labels14_binary.csv")

train test 的 数据都有问题  
需要重新做
基于label的问题已经重新做了数据   
数据位置：  
train--coding/constrastive_P/diagnosisP/exchange/Fine-Grained_Features_Alignment_via_Constrastive_Learning/data/project_using_data/all_train_data_3_11.csv   
test--coding/constrastive_P/diagnosisP/exchange/Fine-Grained_Features_Alignment_via_Constrastive_Learning/data/project_using_data/all_test_3_11.csv   
validate--coding/constrastive_P/diagnosisP/exchange/Fine-Grained_Features_Alignment_via_Constrastive_Learning/data/project_using_data/all_validata_3_11.csv